In [1]:
import pandas as pd
import numpy as np

In [2]:
# 샘플 데이터프레임 생성
idx = ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'C','D', 'D', 'D', 'D', 'E', 'E', 'E']
col = ['col1', 'col2', 'col3']
data = np.random.randint(0,9, (15,3))
df1 = pd.DataFrame(data=data, index=idx, columns = col).reset_index()
df1

,index,col1,col2,col3
0,A,7,8,3
1,A,6,8,1
2,B,2,6,1
3,B,6,5,2
4,B,3,0,3
5,C,6,5,1
6,C,1,1,4
7,C,3,6,0
8,D,6,2,2
9,D,3,8,0


In [3]:
# groupby() : 특정 컬럼별 통계치 구하기
df1.groupby('index').mean()  # sum() 합계 | mean() 평균 | count() 개수 | val() 분산 | std() 표준편차 | max() 최대값 | min() 최소값

,col1,col2,col3
index,,,
A,6.500000,8.000000,2.000000
B,3.666667,3.666667,2.000000
C,3.333333,4.000000,1.666667
D,3.750000,5.250000,3.500000
E,5.000000,4.333333,5.000000


In [4]:
# 둘 이상의 통게치 동시에 구하기
df1.groupby('index').agg(['sum', 'mean'])

col1           col2           col3          
       sum      mean  sum      mean  sum      mean
index                                             
A       13  6.500000   16  8.000000    4  2.000000
B       11  3.666667   11  3.666667    6  2.000000
C       10  3.333333   12  4.000000    5  1.666667
D       15  3.750000   21  5.250000   14  3.500000
E       15  5.000000   13  4.333333   15  5.000000

In [5]:
# 둘 이상의 통게치 동시에 구할 때 컬럼의 멀티인덱스 정리, 소수점도 정리
def flatten_cols(df):
    df.columns = [' / '.join(x) for x in df.columns.to_flat_index()]
    return df

df1.groupby('index').agg(['sum', 'mean']).pipe(flatten_cols).round(2)

,col1 / sum,col1 / mean,col2 / sum,col2 / mean,col3 / sum,col3 / mean
index,,,,,,
A,13,6.50,16,8.00,4,2.00
B,11,3.67,11,3.67,6,2.00
C,10,3.33,12,4.00,5,1.67
D,15,3.75,21,5.25,14,3.50
E,15,5.00,13,4.33,15,5.00


In [6]:
# apply 메서드 활용, 각 그룹별 col1 상위 2위만 추출
def top(df1, n=2, col='col1'):
    return df1.sort_values(by=col)[-n:]
df1.groupby('index', group_keys=False).apply(top)

,index,col1,col2,col3
1,A,6,8,1
0,A,7,8,3
4,B,3,0,3
3,B,6,5,2
7,C,3,6,0
5,C,6,5,1
11,D,5,7,8
8,D,6,2,2
12,E,6,8,6
13,E,7,4,4


In [7]:
# 조금더 쉽게
def get_top2(x):
    return x.sort_values('col1').head(2)
df1.groupby('index').apply(get_top2)

index  col1  col2  col3
index                           
A     1      A     6     8     1
      0      A     7     8     3
B     2      B     2     6     1
      4      B     3     0     3
C     6      C     1     1     4
      7      C     3     6     0
D     10     D     1     4     4
      9      D     3     8     0
E     14     E     2     1     5
      12     E     6     8     6

In [8]:
# lambda 로도 가능
df1.groupby('index').apply(lambda x:x.sort_values('col1').head(2))

index  col1  col2  col3
index                           
A     1      A     6     8     1
      0      A     7     8     3
B     2      B     2     6     1
      4      B     3     0     3
C     6      C     1     1     4
      7      C     3     6     0
D     10     D     1     4     4
      9      D     3     8     0
E     14     E     2     1     5
      12     E     6     8     6

In [9]:
# 기존 인덱스 유지, 아래의 reset_index() 붙인 거와 동일 (그래서 별 의미 없을수도)
df1.groupby('index', as_index=False).sum()

,index,col1,col2,col3
0,A,13,16,4
1,B,11,11,6
2,C,10,12,5
3,D,15,21,14
4,E,15,13,15


In [10]:
df1.groupby('index').sum().reset_index()

,index,col1,col2,col3
0,A,13,16,4
1,B,11,11,6
2,C,10,12,5
3,D,15,21,14
4,E,15,13,15


In [11]:
# Nan 값은 계산에서 제외
df1.loc[6, 'index'] = np.NaN
df1.groupby('index').sum()

,col1,col2,col3
index,,,
A,13,16,4
B,11,11,6
C,9,11,1
D,15,21,14
E,15,13,15


In [12]:
# 하지만 dropna=False 인 경우 인덱스에 Nan 포함되어 계산
df1.groupby('index', dropna=False).sum()

,col1,col2,col3
index,,,
A,13,16,4
B,11,11,6
C,9,11,1
D,15,21,14
E,15,13,15
NaN,1,1,4


In [13]:
df1.groupby('index').agg({
    'col1' : 'sum',
    'col2' : ['sum', 'var']})


col1 col2           
       sum  sum        var
index                     
A       13   16   0.000000
B       11   11  10.333333
C        9   11   0.500000
D       15   21   7.583333
E       15   13  12.333333

In [14]:
# 그룹별로 각각 출력
for idx, group in df1.groupby('index'):
    display(group.head())

,index,col1,col2,col3
0,A,7,8,3
1,A,6,8,1


,index,col1,col2,col3
2,B,2,6,1
3,B,6,5,2
4,B,3,0,3


,index,col1,col2,col3
5,C,6,5,1
7,C,3,6,0


,index,col1,col2,col3
8,D,6,2,2
9,D,3,8,0
10,D,1,4,4
11,D,5,7,8


,index,col1,col2,col3
12,E,6,8,6
13,E,7,4,4
14,E,2,1,5
